# NCML metadata conversion to STAC Item using CWL Application Package annotation

This notebook should be compiled into a standalone *CWL* definition using the following command:

```shell
jupyter-repo2cwl test_ncml2stac.ipynb -o test_ncml2stac.cwl
```

It can then be deployed in *Weaver* using the CLI:

```shell
weaver deploy -u http://example.com/weaver --cwl test_ncml2stac.cwl
```

## Define the CWL Inputs for this Notebook

A real notebook does not even need to import `ipython2cwl` types!
It is sufficient to use only the string typing annotation to avoid import errors when this dependency is not installed!

In [2]:
from typing import TYPE_CHECKING

if TYPE_CHECKING:
    from ipython2cwl.iotypes import CWLFilePathInput, CWLFilePathOutput

input_ncml: "CWLFilePathInput" = "https://raw.githubusercontent.com/crim-ca/stac-populator/ce268cdcde6030b3813f858ab1342b7cafa463e3/tests/data/o3_Amon_GFDL-ESM4_historical_r1i1p1f1_gr1_185001-194912.xml"

## Define the Core functionality of this Notebook

This code assumes that the following reference and all its dependencies are installed:

https://github.com/crim-ca/stac-populator/commit/ce268cdcde6030b3813f858ab1342b7cafa463e3

In [3]:
!rm -fr ~/.esdoc/pyessv-archive
!mkdir -p ~/.esdoc/
!git clone https://github.com/ES-DOC/pyessv-archive ~/.esdoc/pyessv-archive

Cloning into '/home/francis/.esdoc/pyessv-archive'...
remote: Enumerating objects: 63068, done.
remote: Counting objects: 100% (1557/1557), done.6/1557)
remote: Compressing objects: 100% (476/476), done.
remote: Total 63068 (delta 1258), reused 1327 (delta 1070), pack-reused 61511s:  27% (17029/63068), 1.33 MiB | 1.31 MiB/sReceiving objects:  29% (18290/63068), 1.33 MiB | 1.31 MiB/sReceiving objects:  33% (20813/63068), 1.33 MiB | 1.31 MiB/sReceiving objects:  36% (22705/63068), 1.33 MiB | 1.31 MiB/sReceiving objects:  39% (24597/63068), 1.33 MiB | 1.31 MiB/sReceiving objects:  42% (26489/63068), 2.12 MiB | 1.38 MiB/sReceiving objects:  48% (30273/63068), 2.12 MiB | 1.38 MiB/sReceiving objects:  52% (32796/63068), 2.12 MiB | 1.38 MiB/sReceiving objects:  57% (35949/63068), 2.12 MiB | 1.38 MiB/sReceiving objects:  61% (38912/63068), 2.12 MiB | 1.38 MiB/sReceiving objects:  64% (40364/63068), 2.96 MiB | 1.45 MiB/sReceiving objects:  71% (44779/63068), 2.96 MiB | 1.45 MiB/sReceiving objec

In [16]:
import json
import tempfile
from datetime import datetime, date
from dateutil.parser import parse as dt_parse
from enum import Enum

import numpy as np
import requests
import xncml
import pystac
from pystac.extensions import datacube
from pydantic.networks import Url
from shapely.geometry.polygon import Polygon

import pyessv
pyessv.init()

from STACpopulator.extensions import cmip6

# retrieve the file contents
if not (input_ncml.startswith("/") or input_ncml.startswith("file:///")):
    resp = requests.get(input_ncml, headers={"Accept": "text/xml, application/xml"}, timeout=5)
    if not resp.status_code == 200 and resp.text.startswith("<?xml"):
        raise ValueError(f"Could not retrieve NCML XML file contents from [{input_ncml}]. Error: [{resp.status_code}]: {resp.text!s}")
    with tempfile.NamedTemporaryFile(mode="w", encoding="utf-8", delete=False) as tmp_file:
        tmp_file.write(resp.text)
        input_ncml = tmp_file.name

# for debugging purposes, display the contents:
with open(input_ncml, mode="r", encoding="utf-8") as input_ncml_file:
    print(input_ncml_file.read())

2023-09-27 22:47:33.681584 [INFO] :: PYESSV :: Loading vocabularies from /home/francis/.esdoc/pyessv-archive ... please wait
<?xml version="1.0" encoding="UTF-8"?>
<ncml:netcdf xmlns:ncml="http://www.unidata.ucar.edu/namespaces/netcdf/ncml-2.2" location="https://pavics.ouranos.ca/twitcher/ows/proxy/thredds/dodsC/birdhouse/testdata/xclim/cmip6/sic_SImon_CCCma-CanESM5_ssp245_r13i1p2f1_2020.nc">
  <ncml:attribute name="CCCma_model_hash" value="fc4bb7db954c862d023b546e19aec6c588bc0552" />
  <ncml:attribute name="CCCma_parent_runid" value="p2-his13" />
  <ncml:attribute name="CCCma_pycmor_hash" value="26c970628162d607fffd14254956ebc6dd3b6f49" />
  <ncml:attribute name="CCCma_runid" value="p2-s4513" />
  <ncml:attribute name="Conventions" value="CF-1.7 CMIP-6.2" />
  <ncml:attribute name="YMDH_branch_time_in_child" value="2015:01:01:00" />
  <ncml:attribute name="YMDH_branch_time_in_parent" value="2015:01:01:00" />
  <ncml:attribute name="activity_id" value="ScenarioMIP" />
  <ncml:attribute

In [5]:
ds = xncml.Dataset(input_ncml)
attrs = ds.to_cf_dict()
cfmeta = attrs["groups"]["CFMetadata"]["attributes"]
bbox = [
    cfmeta["geospatial_lon_min"][0],
    cfmeta["geospatial_lon_max"][0],
    cfmeta["geospatial_lat_min"][0],
    cfmeta["geospatial_lat_max"][0],
]
geom = Polygon.from_bounds(*bbox)

assets = {
    svc.rsplit("_service", 1)[0]: pystac.Asset(href=svc_link)
    for svc, svc_link in
    attrs["groups"]["THREDDSMetadata"]["groups"]["services"]["attributes"].items()
}

item = pystac.Item(
    id="test",
    start_datetime=dt_parse(cfmeta["time_coverage_start"]),
    end_datetime=dt_parse(cfmeta["time_coverage_end"]),
    datetime=None,  # uses start/end instead
    bbox=bbox,
    geometry=geom.__geo_interface__,  # GeoJSON
    properties={},  # filled by extension after
    assets=assets,
)

dc_dims = {}
dim_spatial_axis = {
    "i": "x",
    "j": "y",
    "k": "z",
    "x": "x",
    "y": "y",
    "z": "z",
}
for dim, val in attrs["dimensions"].items():
    dim_props = {
        "type": (
            datacube.DimensionType.SPATIAL if dim in list(dim_spatial_axis) else
            datacube.DimensionType.TEMPORAL if dim in ["date", "time", "date-time", "datetime"] else
            "other"
        ),
        "length": val
    }
    if dim_props["type"] == datacube.DimensionType.SPATIAL.value:
        dim_props["axis"] = dim_spatial_axis[dim]
    dc_dims[dim] = datacube.Dimension(dim_props)

dc_vars = {}
for var, props in attrs["variables"].items():
    var_dims = props["shape"]
    var_props = {
        "type": (
            datacube.VariableType.DATA
            if all(_dim in dc_dims for _dim in var_dims)
            else datacube.VariableType.AUXILIARY
        ),
        "dimensions": var_dims,
    }
    var_unit = props.get("attributes", {}).get("units")
    if var_unit:
        var_props["unit"] = var_unit
    dc_vars[var] = datacube.Variable(var_props)

datacube_ext = datacube.DatacubeExtension.ext(item, add_if_missing=True)
datacube_ext.apply(dimensions=dc_dims, variables=dc_vars)

cmip6_attrs = attrs["attributes"]
cmip6_ext = cmip6.CMIP6Extension.ext(item, add_if_missing=True)
cmip6_ext.apply(cmip6_attrs)

/home/francis/dev/conda/envs/weaver/lib/python3.10/site-packages/xncml/core.py:482: UserWarning: Could not cast date:
'modified' is not a valid DataType
  warn(f"Could not cast {attr['@name']}:\n{exc}")


## Display the result for validation

In [6]:
stac_item = item.to_dict()
stac_item

{'type': 'Feature',
 'stac_version': '1.0.0',
 'id': 'test',
 'properties': {'start_datetime': None,
  'end_datetime': None,
  'cube:dimensions': {'time': {'type': <DimensionType.TEMPORAL: 'temporal'>,
    'length': 12},
   'bnds': {'type': 'other', 'length': 2},
   'vertices': {'type': 'other', 'length': 4},
   'maxStrlen64': {'type': 'other', 'length': 64},
   'j': {'type': <DimensionType.SPATIAL: 'spatial'>,
    'length': 291,
    'axis': 'y'},
   'i': {'type': <DimensionType.SPATIAL: 'spatial'>,
    'length': 360,
    'axis': 'x'}},
  'cube:variables': {'time': {'type': <VariableType.DATA: 'data'>,
    'dimensions': ['time'],
    'unit': 'days since 1850-01-01'},
   'j': {'type': <VariableType.DATA: 'data'>,
    'dimensions': ['j'],
    'unit': '1'},
   'i': {'type': <VariableType.DATA: 'data'>,
    'dimensions': ['i'],
    'unit': '1'},
   'time_bnds': {'type': <VariableType.DATA: 'data'>,
    'dimensions': ['time', 'bnds']},
   'vertices_latitude': {'type': <VariableType.DATA: 'd

## Define the CWL Outputs for this Notebook

In [17]:
def json_encode(obj):
    if isinstance(obj, (np.ndarray, np.number)):
        return obj.tolist()
    if isinstance(obj, (Url, Enum)):
        return str(obj)
    if isinstance(obj, (datetime, date)):
        return obj.isoformat()
    raise TypeError(f"Type {type(obj)} not serializable")

with tempfile.NamedTemporaryFile(mode="w", encoding="utf-8", delete=False) as out_file:
    json.dump(stac_item, out_file, default=json_encode)

output: "CWLFilePathOutput" = out_file.name